In [375]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("Fields_lessColumns.csv")

# df = df.loc[df['locality'] == 'Bucuresti']

del df['_id'] 
del df['locality']
del df['Unnamed: 0'] # dafaq is this column? delete it...
del df['Unnamed: 0.1']

# del df['district']
del df['landType']
del df['landClassification']
# del df['frontStradal']
# del df['noFronturi']

df["landArea"] = pd.to_numeric(df["landArea"], errors='coerce')
df = df.loc[df['landArea'] > 1000]

print(f'Before droping rows with NaN values: {len(df)}')
df.dropna(inplace=True)
print(f'After dropping rows with NaN {len(df)}')

target = df['price']
del df['price']

# districts_one_hot = pd.get_dummies(df.district)
# df = df.join(districts_one_hot)
del df["district"]

df['contructionOnLand'] = df['contructionOnLand'].replace({
    "Da":1,
    "Nu": 0
})

X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2)


Before droping rows with NaN values: 10089
After dropping rows with NaN 1584


In [376]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_test


,landArea,frontStradal,noFronturi,contructionOnLand
4562,2516.0,30.00,1.0,0
4909,14320.0,142.00,2.0,0
224,1800.0,22.70,2.0,0
4154,5400.0,43.00,2.0,0
4259,7965.0,26.00,2.0,0
...,...,...,...,...
1563,14000.0,73.00,2.0,0
501,8440.0,46.00,1.0,0
1053,2360.0,94.00,2.0,0
3616,13000.0,25.00,1.0,0


In [377]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=3)
X_train = poly_features.fit_transform(X_train)

reg = LinearRegression().fit(X_train, y_train.values)

In [378]:
X_test = poly_features.fit_transform(X_test)
print(reg.score(X_train, y_train), reg.score(X_test, y_test))
for i in range(10):
    print(y_test.values[i] , reg.predict([X_test[i]]))

0.30696320665076693 0.08605384203494382
129800.0 [270595.30150662]
7160000.0 [762252.17047033]
32000.0 [233529.89660421]
648000.0 [361862.40952277]
310635.0 [351194.83925515]
15999.0 [212358.76977205]
105000.0 [296658.34460274]
1322500.0 [464386.18883985]
275000.0 [482139.99647194]
129800.0 [270595.30150662]


### Random Forest

In [379]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [380]:
print(rf.score(X_train, y_train), rf.score(X_test, y_test))
for i in range(10):
    print(y_test.values[i] , rf.predict([X_test[i]]), y_test.values[i] - rf.predict([X_test[i]]))

0.8586614277485027 0.19227800000287543
129800.0 [140029.469] [-10229.469]
7160000.0 [936435.31802] [6223564.68198]
32000.0 [120791.97007] [-88791.97007]
648000.0 [629766.52131] [18233.47869]
310635.0 [310040.71113333] [594.28886667]
15999.0 [56359.87928571] [-40360.87928571]
105000.0 [351974.28284] [-246974.28284]
1322500.0 [539258.34827619] [783241.65172381]
275000.0 [411576.84066667] [-136576.84066667]
129800.0 [140029.469] [-10229.469]


### XGBoost

In [382]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [385]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
print(y_pred)
predictions = [round(value) for value in y_pred]

for i in range(10):
    print(y_test.values[i] , predictions[i], y_test.values[i] - predictions[i])

[1.29800e+05 1.05000e+06 1.76750e+05 7.50000e+05 1.00000e+06 2.20000e+04
 3.50000e+04 9.90000e+05 2.36712e+05 1.29800e+05 1.22160e+05 7.50000e+04
 4.20000e+04 9.50000e+04 2.07000e+05 9.00000e+04 1.25000e+05 8.00000e+05
 1.08000e+05 4.20000e+04 4.60000e+04 9.00000e+05 2.00000e+05 2.88000e+05
 1.50000e+05 1.50000e+05 9.00000e+04 4.50000e+04 9.60000e+04 8.00000e+04
 1.11300e+05 1.93500e+06 8.00000e+05 4.00000e+05 6.50000e+05 1.20000e+05
 1.80000e+04 4.20000e+04 1.90000e+05 3.00000e+05 4.58800e+05 8.00000e+05
 9.00000e+04 6.22500e+05 1.80000e+04 4.00000e+05 2.25000e+05 9.10000e+05
 3.90000e+06 3.00000e+04 2.60000e+05 2.40000e+04 6.30000e+05 2.40000e+04
 9.00000e+04 2.88000e+05 4.25000e+04 3.90000e+04 1.20000e+05 2.40000e+04
 1.31200e+05 1.62000e+05 4.50000e+04 9.90000e+05 3.80000e+04 2.22500e+05
 7.50000e+04 8.10000e+04 4.00000e+05 4.50000e+04 1.30000e+05 3.29000e+06
 2.20000e+05 6.00000e+04 7.35000e+05 1.36000e+05 2.00000e+05 9.99000e+05
 9.00000e+04 6.00000e+05 9.00000e+04 9.00000e+04 4.

### Encoders tests

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)
enc.categories_

In [23]:
enc.transform([['Female', 1], ['Male', 4]]).toarray()

array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [25]:
enc.get_feature_names(['gender', 'plm'])

array(['gender_Female', 'gender_Male', 'plm_1', 'plm_2', 'plm_3'],
      dtype=object)